In [43]:
import os
from tensorflow.python.platform import gfile
import scipy.io.wavfile as wav
from python_speech_features import *
import numpy as np
from subprocess import call
from struct import unpack, pack

DATA_DIR = os.path.join("/n/sd7/trung/csp", "data", "vivos")

In [51]:
mode = "train"

wset = set()
with open(os.path.join(DATA_DIR, mode, "prompts.txt"), encoding="utf-8") as f:
    for s in f.read().split('\n'):
        words = s.split(' ')[1:]
        for word in words: wset.add(word.lower())

print(len(wset))
with open(os.path.join(DATA_DIR, "words.txt"), "w", encoding="utf-8") as f:
    f.write("<unk> 0\n<sos> 1\n<eos> 2\n")
    f.write("\n".join(["%s %d" % (word, i + 2) for i, word in
        enumerate(list(wset)) if word != ""]))

4862
[]


In [19]:
# get mean
mean = np.array([0] * 120)
var = np.array([0] * 120)
count = 0
with open(os.path.join(DATA_DIR, "train", "prompts.txt"), encoding="utf-8") as f:
    lines = f.read().split("\n")
    for i, s in enumerate(lines):
        filename = s.split(' ')[0]
        if filename == "": continue
        d[filename] = ' '.join(s.split(' ')[1:])
        wav_filename = os.path.join(DATA_DIR, "train", "waves", filename.split('_')[0], filename + ".wav")
        npy_filename = os.path.join(DATA_DIR, "train", "npy", filename.split('_')[0], filename + ".npy")
        (rate, sig) = wav.read(wav_filename)

        filename = os.path.join(DATA_DIR, "train", "feature", filename + ".htk")
        call([
            "/n/sd7/trung/bin/htk/HTKTools/HCopy",
            wav_filename,
            filename,
            "-C", "/n/sd7/trung/config.lmfb.40ch"
        ])
        fh = open(filename, "rb")
        spam = fh.read(12)
        nSamples, sampPeriod, sampSize, parmKind = unpack(">IIHH", spam)
        veclen = int(sampSize / 4)
        fh.seek(12, 0)
        dat = np.fromfile(fh, dtype=np.float32)
        dat = dat.reshape(len(dat) // veclen, veclen)
        dat = dat.byteswap()
        fh.close()
        
        for k in range(len(dat)):
            updated_mean = (mean * count + np.sum(dat, axis=0)) / (count + 1)
            var = (count * var + (dat[k] - mean) * (dat[k] - updated_mean)) / (count + 1)
            mean = updated_mean
            count += 1

        outputs.append(
            (npy_filename,
            ' '.join(['2'] + [words[w.lower()] if w.lower() in words else \
                '0' for w in s.split(' ')[1:]] + ['1']))
        )
            
        if i % 100 == 1: print("%d / %d" % (i, len(lines)))
            
np.save(os.path.join(DATA_DIR, "mean.npy"), mean)
np.save(os.path.join(DATA_DIR, "var.npy"), var)
print(mean)

1 / 11661
101 / 11661
201 / 11661
301 / 11661
401 / 11661
501 / 11661
601 / 11661
701 / 11661
801 / 11661
901 / 11661
1001 / 11661
1101 / 11661
1201 / 11661
1301 / 11661
1401 / 11661
1501 / 11661
1601 / 11661
1701 / 11661
1801 / 11661
1901 / 11661
2001 / 11661
2101 / 11661
2201 / 11661
2301 / 11661
2401 / 11661
2501 / 11661
2601 / 11661
2701 / 11661
2801 / 11661
2901 / 11661
3001 / 11661
3101 / 11661
3201 / 11661
3301 / 11661
3401 / 11661
3501 / 11661
3601 / 11661
3701 / 11661
3801 / 11661
3901 / 11661
4001 / 11661
4101 / 11661
4201 / 11661
4301 / 11661
4401 / 11661
4501 / 11661
4601 / 11661
4701 / 11661
4801 / 11661
4901 / 11661
5001 / 11661
5101 / 11661
5201 / 11661
5301 / 11661
5401 / 11661
5501 / 11661
5601 / 11661
5701 / 11661
5801 / 11661
5901 / 11661
6001 / 11661
6101 / 11661
6201 / 11661
6301 / 11661
6401 / 11661
6501 / 11661
6601 / 11661
6701 / 11661
6801 / 11661
6901 / 11661
7001 / 11661
7101 / 11661
7201 / 11661
7301 / 11661
7401 / 11661
7501 / 11661
7601 / 11661
7701 / 1166

In [58]:
mean = np.load(os.path.join(DATA_DIR, "mean.npy"))
var = np.load(os.path.join(DATA_DIR, "var.npy"))

def get_features_npy(infile, outfile):
    fh = open(infile, "rb")
    spam = fh.read(12)
    nSamples, sampPeriod, sampSize, parmKind = unpack(">IIHH", spam)
    veclen = int(sampSize / 4)
    fh.seek(12, 0)
    dat = np.fromfile(fh, dtype=np.float32)
    dat = dat.reshape(len(dat) // veclen, veclen)
    dat = dat.byteswap()
    fh.close()
    
    dat = (dat - mean) / np.sqrt(var)
    print(len(dat), dat)
    np.save(outfile, dat)

In [59]:
mode = "test"

d = {}
words = open(os.path.join(DATA_DIR, "words.txt")).read().split('\n')
words = { word.split(' ')[0]: word.split(' ')[1] for word in words if word != ""}
outputs = []
with open(os.path.join(DATA_DIR, mode, "prompts.txt"), encoding="utf-8") as f:
    lines = f.read().split("\n")
    for i, s in enumerate(lines):
        filename = s.split(' ')[0]
        if filename == "": continue
        d[filename] = ' '.join(s.split(' ')[1:])
        wav_filename = os.path.join(DATA_DIR, mode, "waves", filename.split('_')[0], filename + ".wav")
        npy_filename = os.path.join(DATA_DIR, mode, "npy", filename + ".npy")
        (rate, sig) = wav.read(wav_filename)

        filename = os.path.join(DATA_DIR, mode, "feature", filename + ".htk")
        #call([
        #    "/n/sd7/trung/bin/htk/HTKTools/HCopy",
        #    "-C", "/n/sd7/trung/config.lmfb.40ch", "-T", "1",
        #    wav_filename, filename
        #])
        get_features_npy(filename, npy_filename)

        outputs.append(
            (npy_filename,
            ' '.join(['2'] + [words[w.lower()] if w.lower() in words else \
                '0' for w in s.split(' ')[1:]] + ['1']))
        )
            
        if i % 100 == 1: print("%d / %d" % (i, len(lines)))

outputs.sort(key=lambda x: len(x[1]))
with open(os.path.join(DATA_DIR, mode, "data.txt"), "w", encoding="utf-8") as f:
        f.write('\n'.join(["%s %s" % (fn, tg) for fn, tg in outputs]))

261 [[-1.17288827 -1.1884899  -1.20976886 ...  0.08215611 -0.17961475
  -0.03256121]
 [-1.1727526  -1.18840039 -1.20986658 ...  0.22763364 -0.32621942
  -0.09676652]
 [-1.17275972 -1.18837557 -1.21020894 ...  0.42469402 -0.35088136
  -0.07317104]
 ...
 [-1.17295726 -1.18865163 -1.21000401 ... -0.02878575  0.22498923
  -0.12311165]
 [-1.17292958 -1.18846277 -1.20987905 ...  0.04041412  0.09947199
  -0.01481743]
 [-1.1727409  -1.18834693 -1.20980599 ...  0.0482264  -0.01681819
  -0.00470565]]
254 [[-1.17317715 -1.18872448 -1.20988737 ... -0.02690029 -0.21851271
  -0.1402555 ]
 [-1.17300381 -1.18867714 -1.20964218 ...  0.07401406 -0.29217526
  -0.08607178]
 [-1.17282513 -1.18847905 -1.20993443 ...  0.01534018 -0.17737283
   0.08327529]
 ...
 [-1.17335165 -1.18861236 -1.2098013  ... -0.14867868 -0.03766903
   0.01656129]
 [-1.17278655 -1.18829821 -1.20997832 ... -0.10927812 -0.00306768
   0.18103611]
 [-1.17283061 -1.18849561 -1.20971141 ... -0.05318607 -0.05039218
   0.07835359]]
1 / 761


KeyboardInterrupt: 